In [1]:
import os
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-12-01 14:01:06--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.47MB/s    in 0.2s    

2021-12-01 14:01:06 (5.47 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Hashing").getOrCreate()

In [4]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("header","true").csv(SparkFiles.get("amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz"), inferSchema=True, sep="\t")

# Show DataFrame
df.show(truncate=True)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33605939| RGYFDX8QXKEIR|B007KO2MLO|     328837464|           Big Maria|Digital_Ebook_Pur...|          4|            0|          0|   N|                N|              Quirky|Elmore Leonard me...| 2013-09-09|
|         US|   34058393|R13CBGTMNV9R8Z|B005FLODDE|     764276359|The Woman Who Was...|Digital_Ebook_Pur

In [6]:
# Count the number of records (rows) in the dataset.
df.count()

5101693

In [7]:
## Transform the dataset to fit the tables in the schema file. Be sure the DataFrames match in data type and in column name.

In [8]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [9]:
# Review ID Table Creation
# CREATE TABLE review_id_table (
#   review_id TEXT PRIMARY KEY NOT NULL,
#   customer_id INTEGER,
#   product_id TEXT,
#   product_parent INTEGER,
#   review_date DATE -- this should be in the formate yyyy-mm-dd
# );

# Review_date in this table will need to be cast as a date to match the appropriate schema provided.

review_id_table = df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RGYFDX8QXKEIR|   33605939|B007KO2MLO|     328837464| 2013-09-09|
|R13CBGTMNV9R8Z|   34058393|B005FLODDE|     764276359| 2013-09-09|
| R7DRFHC0F71O0|   39601147|B00EA3L35O|     535606445| 2013-09-09|
|R27LUKEXU3KBXQ|   17351407|B00BL3JV50|     240053004| 2013-09-09|
|R1VXTPUYMNU687|   10463387|B00CXU7U80|     931529805| 2013-09-09|
|R30DKW1GJWLPZC|   50484904|B004EWGS5G|     442453110| 2013-09-09|
|R18DPFG2FALJI9|    7145636|B00BNRJAT6|     856774152| 2013-09-09|
|R24D677N5WBW5Q|    6285538|B007FZOXJM|       5589837| 2013-09-09|
|R2FCJ9BQLSIOR3|   10278048|B00B6AK7LU|     362701357| 2013-09-09|
|R1R6K4MAKDWTXI|   16568972|B00EVMMLU0|     342745087| 2013-09-09|
|R3R5DILCWM8J7B|   26001763|B00E4W4984|     186117550| 2013-09-09|
| RR5K72IZOCOFE|   48203259|B005A1JBB8|     947574172| 2013-09

In [10]:
from pyspark.sql.types import * 
review_id_table = review_id_table.withColumn("review_date",review_id_table["review_date"].cast(DateType()))

In [11]:
# -- This table will contain only unique values
# CREATE TABLE products (
#   product_id TEXT PRIMARY KEY NOT NULL UNIQUE,
#   product_title TEXT
# );

# No data types in the desired table are different then the original DF. Moving on!

products = df.select(['product_id', 'product_title'])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B007KO2MLO|           Big Maria|
|B005FLODDE|The Woman Who Was...|
|B00EA3L35O|Mary had A Sleepy...|
|B00BL3JV50|          Starstruck|
|B00CXU7U80|The Complete Cona...|
|B004EWGS5G|The Middle Passag...|
|B00BNRJAT6|Hide in Plain Sig...|
|B007FZOXJM|Face of Betrayal ...|
|B00B6AK7LU|Final Justice (A ...|
|B00EVMMLU0|Falling For My Hu...|
|B00E4W4984|  Forgetting Tabitha|
|B005A1JBB8|The Blacksmith's ...|
|B0044DELIQ|Vampire Vacation:...|
|B00DJQYATE|By the Tail (Coug...|
|B004XFYNNS|The Kingdom (A Fa...|
|B00A27Z6GO|6 Erotic Bedtime ...|
|B008EGV4BQ|The Purpose Drive...|
|B00655U3WE|Insurgent (Diverg...|
|B006NG0GXQ|Forever His: A Ti...|
|B008J4RPWA|Summerset Abbey: ...|
+----------+--------------------+
only showing top 20 rows



In [12]:
# -- Customer table for first data set
# CREATE TABLE customers (
#   customer_id INT PRIMARY KEY NOT NULL UNIQUE,
#   customer_count INT
# );

# I know customer_id is an integer, but don't know with 100% certainty what data type the agg function will populate into customer_count. 

customers = df.groupby('customer_id').agg({'customer_id':'count'}).withColumnRenamed('count(customer_id)', 'customer_count')
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   27134317|           306|
|   36063094|            15|
|   49422311|             1|
|   18978114|             1|
|   13455482|             2|
|   32153206|             1|
|   10274872|             4|
|   20977654|             1|
|   48042607|             1|
|   11376183|             7|
|   31762963|             1|
|   11481508|             4|
|   12362676|             2|
|   31176171|            10|
|   44699860|            23|
|   52734389|             8|
|   24303040|             3|
|   26322451|            77|
|   14521576|            30|
|   44269342|            10|
+-----------+--------------+
only showing top 20 rows



In [13]:
# I know from the previous notebook that the .agg() function will return long, and I need the data type to be integer. 
customers = customers.withColumn("customer_count ",customers["customer_count"].cast(IntegerType()))

In [14]:
# -- vine table
# CREATE TABLE vine_table (
#   review_id TEXT PRIMARY KEY,
#   star_rating INTEGER,
#   helpful_votes INTEGER,
#   total_votes INTEGER,
#   vine TEXT
# );

# All of the data types in the original DF match the schem provided to create this table, so no casting will be necessary.

vine_table = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RGYFDX8QXKEIR|          4|            0|          0|   N|
|R13CBGTMNV9R8Z|          4|            1|          2|   N|
| R7DRFHC0F71O0|          5|            0|          0|   N|
|R27LUKEXU3KBXQ|          5|            1|          1|   N|
|R1VXTPUYMNU687|          5|            1|          2|   N|
|R30DKW1GJWLPZC|          3|            1|          2|   N|
|R18DPFG2FALJI9|          5|            0|          0|   N|
|R24D677N5WBW5Q|          5|            0|          0|   N|
|R2FCJ9BQLSIOR3|          5|            0|          0|   N|
|R1R6K4MAKDWTXI|          4|            0|          0|   N|
|R3R5DILCWM8J7B|          5|            0|          0|   N|
| RR5K72IZOCOFE|          4|            0|          0|   N|
|R3K9PJU5GLDY3O|          5|            1|          2|   N|
|R1KTZMCDOJXAEK|          5|            

In [15]:
# Pushing all these tables up to an RDS instance
mode = "append"
jdbc_url = "jdbc:postgresql://<connection>/<dbname>"
config =  {"user":"<user>",
         "password":"<password>",
         "driver":"org.postrgresql.Driver"}

In [ ]:
# Table 1
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Table 2
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Table 3
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Table 4
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)